## Düzgün görmek için vs code'a jupyter paketini yüklemeniz lazım

In [1]:
import pyomo.environ as pyo
import pandas as pd

In [2]:
#!pip install openpyxl

directory="Scenarios\ProjectPart1-Scenario2.xlsx"
#pandas depens on this package to read excel files
orders=pd.read_excel(directory,sheet_name="Orders")
orders

,Order ID,Product Type,Demand Amount,Due Date,Earliness Penalty
0,1,P2,175,1,25
1,1,P3,145,1,25
2,1,P4,130,1,25
3,1,P7,190,1,25
4,1,P8,145,1,25
...,...,...,...,...,...
149,30,P4,140,1,10
150,30,P5,105,1,10
151,30,P7,145,1,10
152,30,P8,125,1,10


In [3]:
trucks=pd.read_excel(directory,sheet_name="Vehicles")
trucks

,Vehicle ID,Vehicle Type,Capacity for pallet type 1,Capacity for pallet type 2,Fixed Cost (c_k),Variable Cost (c'_k)
0,1,1,22,33,100,150
1,2,2,12,18,80,120
2,3,3,6,8,50,75
3,4,1,22,33,100,150
4,5,2,12,18,80,120
5,6,3,6,8,50,75
6,7,1,22,33,100,150
7,8,2,12,18,80,120
8,9,3,6,8,50,75
9,10,2,12,18,80,120


In [4]:
pallets=pd.read_excel(directory,sheet_name="Pallets")
pallets

,Pallet ID,Product Type,Amount,Pallet Size,Release Day
0,1,P1,65,2,1
1,2,P1,80,2,1
2,3,P1,70,2,1
3,4,P1,50,2,1
4,5,P1,50,2,1
...,...,...,...,...,...
297,298,P9,85,2,7
298,299,P9,70,2,7
299,300,P9,80,2,7
300,301,P9,80,2,7


In [5]:
pallets["Product Type"].unique()

array(['P1', 'P2', 'P3', 'P4', 'P5', 'P7', 'P8', 'P9', 'P10', 'P6'],
      dtype=object)

In [6]:
extraParam=pd.read_excel(directory,sheet_name="Parameters")
extraParam

,Parameter,Value
0,Planning Horizon (T),7
1,Max. trips per period,3
2,Max. pallets in area,15


In [7]:
# --------------------------------------------------------
# 1) Create the model
# --------------------------------------------------------
model = pyo.ConcreteModel()


# --------------------------------------------------------
# 2) Define Sets
# --------------------------------------------------------
model.I = pyo.Set(initialize=range(1, len(pallets)+1), doc="Set of pallets")
model.K = pyo.Set(initialize=trucks["Vehicle Type"].unique().tolist(), doc="Set of vehicle types")

# Don't forget to change the indexes if there is a change in data, default order:
# 0	Planning Horizon (T)
# 1	Max. trips per period	
# 2	Max. pallets in area

model.T = pyo.Set(initialize=range(1, extraParam.iloc[0,1]+1), doc="Set of days")
model.M = pyo.Set(initialize=range(1, extraParam.iloc[1,1]+1), doc="Set of possible trips per day")

model.S = pyo.Set(initialize=range(1, pallets["Pallet Size"].max()+1), doc="Set of pallet sizes")
model.O = pyo.Set(initialize=orders["Order ID"].unique() ,doc="Set of orders")
# model.J = pyo.Set(initialize=range(1, int(sorted(pallets["Product Type"].unique())[-1][-1])+1), doc="Set of products") 
model.J = pyo.Set(initialize=pallets["Product Type"].unique(), doc="Set of products") 

model.I.display(), model.K.display(), model.T.display(), model.M.display(), model.S.display(), model.O.display(), model.J.display()

I : Set of pallets
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :  302 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194

(None, None, None, None, None, None, None)

In [8]:
capacity_dict = {}
for row in trucks["Vehicle Type"].unique():
    row=trucks[trucks["Vehicle Type"]==row].iloc[0,:]
    k=int(row.iloc[1])
    s=1
    capacity_dict[(k, s)] = row.iloc[2]
    k=int(row.iloc[1])
    s=2
    capacity_dict[(k, s)] = row.iloc[3]

demand_dict = {}
for row in orders.iterrows():
    o=row[1].iloc[0]
    j=row[1].iloc[1]
    demand_dict[(o, j)] = row[1].iloc[2]

In [9]:
# --------------------------------------------------------
# 3) Define Parameters
# --------------------------------------------------------
model.release_day = pyo.Param(model.I, initialize=dict(enumerate(pallets["Release Day"], start=1)), within=pyo.NonNegativeIntegers, doc="r_i: Release day for pallet i")
model.c_owned     = pyo.Param(model.K, initialize=dict(enumerate(trucks.groupby(["Vehicle Type","Fixed Cost (c_k)"]).count().reset_index().iloc[:,1], start=1)), within=pyo.NonNegativeReals, doc="c_k: Cost of using an owned vehicle type k")
model.c_rented    = pyo.Param(model.K, initialize=dict(enumerate(trucks.groupby(["Vehicle Type","Variable Cost (c'_k)"]).count().reset_index().iloc[:,1], start=1)), within=pyo.NonNegativeReals, doc="c'_k: Cost of renting vehicle type k")

#!!!!!!! Unchecked initialization
model.h           = pyo.Param(model.O, initialize=orders.groupby(["Order ID","Earliness Penalty"]).count().reset_index().iloc[:,0:2].set_index("Order ID").to_dict()["Earliness Penalty"], within=pyo.NonNegativeReals, doc="h_o: Per-day earliness penalty for order o")
model.d_due       = pyo.Param(model.O, initialize=orders.groupby(["Order ID","Due Date"]).count().reset_index().iloc[:,0:2].set_index("Order ID").to_dict()["Due Date"], within=pyo.NonNegativeIntegers, doc="d_o: Due day for order o")


model.p_demand    = pyo.Param(model.O, model.J, initialize=demand_dict, default=0, within=pyo.NonNegativeIntegers, doc="p_{o,j}: Demand for product j in order o")
demand_dict=None

model.j_of_i      = pyo.Param(model.I, initialize=dict(enumerate(pallets["Product Type"], start=1)), within=model.J, doc="j(i): Product type of pallet i")
model.n_i         = pyo.Param(model.I, initialize=dict(enumerate(pallets["Amount"], start=1)), within=pyo.NonNegativeIntegers, doc="n_i: Product capacity of pallet i")


model.capacity    = pyo.Param(model.K, model.S, initialize=capacity_dict, within=pyo.NonNegativeIntegers, doc="Capacity_{k,s}: # of pallets of size s that fit in vehicle k")
capacity_dict=None

model.b_k         = pyo.Param(model.K, initialize=dict(enumerate(trucks.groupby("Vehicle Type").count().reset_index().iloc[:,1], start=1)), within=pyo.NonNegativeIntegers, doc="b_k: Number of owned vehicles of type k available")

model.q           = pyo.Param(initialize=extraParam.iloc[2,1],within=pyo.NonNegativeIntegers, doc="q: Max number of pallets allowed in waiting area overnight")

model.release_day.display(), model.h.display(), model.d_due.display(), model.p_demand.display(), model.j_of_i.display(), model.n_i.display(), model.capacity.display(), model.q.display()

release_day : r_i: Release day for pallet i
    Size=302, Index=I, Domain=NonNegativeIntegers, Default=None, Mutable=False
    Key : Value
      1 :     1
      2 :     1
      3 :     1
      4 :     1
      5 :     1
      6 :     1
      7 :     1
      8 :     1
      9 :     1
     10 :     1
     11 :     1
     12 :     1
     13 :     1
     14 :     1
     15 :     1
     16 :     1
     17 :     1
     18 :     1
     19 :     1
     20 :     1
     21 :     1
     22 :     1
     23 :     1
     24 :     1
     25 :     1
     26 :     1
     27 :     1
     28 :     1
     29 :     1
     30 :     1
     31 :     1
     32 :     1
     33 :     1
     34 :     1
     35 :     1
     36 :     1
     37 :     1
     38 :     1
     39 :     1
     40 :     1
     41 :     1
     42 :     1
     43 :     1
     44 :     1
     45 :     1
     46 :     1
     47 :     1
     48 :     1
     49 :     1
     50 :     1
     51 :     1
     52 :     1
     53 :     1
     54 :    

(None, None, None, None, None, None, None, None)

In [10]:

# --------------------------------------------------------
# 4) Define Decision Variables
# --------------------------------------------------------
# 1) Shipment Variables
model.u = pyo.Var(model.K, model.T, model.M, model.S, within=pyo.Binary,
                  doc="u_{k,t,m,s}: 1 if a shipment of vehicle type k on day t, trip m, uses pallet size s")
model.v = pyo.Var(model.K, model.T, model.M, model.S, within=pyo.NonNegativeIntegers,
                  doc="v_{k,t,m,s}: Number of pallets of size s in shipment (k,t,m)")

# 2) Vehicle Usage
model.y = pyo.Var(model.K, model.T, model.M, within=pyo.Binary,
                  doc="y_{k,t,m}: 1 if an owned vehicle of type k is used on day t, trip m")
model.z = pyo.Var(model.K, model.T, model.M, within=pyo.Binary,
                  doc="z_{k,t,m}: 1 if a rented vehicle of type k is used on day t, trip m")

# 3) Pallet Assignment
model.x = pyo.Var(model.I, model.K, model.T, model.M, model.S, within=pyo.Binary,
                  doc="x_{i,k,t,m,s}: 1 if pallet i is shipped via (k,t,m,s)")

# 4) Order Fulfillment
model.e = pyo.Var(model.I, model.O, model.T, within=pyo.NonNegativeIntegers,
                  doc="e_{i,o,t}: products from pallet i allocated to order o and shipped on day t")

# --------------------------------------------------------
# 5) Define Constraints
# --------------------------------------------------------

# (1) Each pallet is shipped exactly once, after its release day
def single_shipment_rule(model, i):
    return sum(model.x[i, k, t, m, s]
               for k in model.K
               for t in model.T if t >= model.release_day[i]
               for m in model.M
               for s in model.S) == 1
model.single_shipment = pyo.Constraint(model.I, rule=single_shipment_rule)


# (2) Shipment Capacity:
#     sum_i x_{i,k,t,m,s} = v_{k,t,m,s} <= capacity_{k,s} * u_{k,t,m,s}
# We can express this in two separate constraints:
def capacity_link_1_rule(model, k, t, m, s):
    # sum of pallets assigned = v_{k,t,m,s}
    return sum(model.x[i, k, t, m, s] for i in model.I) == model.v[k, t, m, s]
model.capacity_link_1 = pyo.Constraint(model.K, model.T, model.M, model.S, rule=capacity_link_1_rule)

def capacity_link_2_rule(model, k, t, m, s):
    # v_{k,t,m,s} <= capacity_{k,s} * u_{k,t,m,s}
    return model.v[k, t, m, s] <= model.capacity[k, s] * model.u[k, t, m, s]
model.capacity_link_2 = pyo.Constraint(model.K, model.T, model.M, model.S, rule=capacity_link_2_rule)


# (3) Single pallet size per shipment
def single_size_rule(model, k, t, m):
    return sum(model.u[k, t, m, s] for s in model.S) <= 1
model.single_size = pyo.Constraint(model.K, model.T, model.M, rule=single_size_rule)


# (4) Vehicle Type usage constraint
#     y_{k,t,m} + z_{k,t,m} = sum_s u_{k,t,m,s}
def vehicle_type_rule(model, k, t, m):
    return model.y[k, t, m] + model.z[k, t, m] == sum(model.u[k, t, m, s] for s in model.S)
model.vehicle_type = pyo.Constraint(model.K, model.T, model.M, rule=vehicle_type_rule)


# (5) Owned Vehicle Daily Trip Limit
#     sum_m y_{k,t,m} <= 3 * b_k
# Here we assume each type k can do up to 3 trips per owned vehicle. Adapt if needed.
def owned_vehicle_limit_rule(model, k, t):
    return sum(model.y[k, t, m] for m in model.M) <= 3 * model.b_k[k]
model.owned_vehicle_limit = pyo.Constraint(model.K, model.T, rule=owned_vehicle_limit_rule)


# (6) Order Demand:
#     sum_{i: j(i)=j} sum_{t <= d_o} e_{i,o,t} >= p_{o,j}
# We iterate over each order o and each product j
# If p_demand[o,j] > 0, we must satisfy that demand by the due date.
def order_demand_rule(model, o, j):
    # sum of allocated products across all pallets i of type j
    return sum(model.e[i, o, t]
               for i in model.I
               if model.j_of_i[i] == j
               for t in model.T
               if t <= model.d_due[o]) >= model.p_demand[o, j]
model.order_demand = pyo.Constraint(model.O, model.J, rule=order_demand_rule)


# (7) Product allocation per pallet:
#     sum_o e_{i,o,t} <= n_i * sum_{k,m,s} x_{i,k,t,m,s}
def product_alloc_rule(model, i, t):
    return sum(model.e[i, o, t] for o in model.O) <= \
           model.n_i[i] * sum(model.x[i, k, t, m, s] for k in model.K for m in model.M for s in model.S)
model.product_alloc = pyo.Constraint(model.I, model.T, rule=product_alloc_rule)


# (8) Waiting Area Limit:
#     For each day t, the number of pallets that have been released but not shipped by day t cannot exceed q
#     sum_{i: r_i <= t} (1 - sum_{t' <= t, k,m,s} x_{i,k,t',m,s}) <= q
def waiting_area_rule(model, t):
    # Only consider pallets i whose release_day[i] <= t
    return sum(
        1 - sum(model.x[i, k, t_prime, m, s]
                for t_prime in model.T if t_prime <= t
                for k in model.K
                for m in model.M
                for s in model.S)
        for i in model.I
        if model.release_day[i] <= t
    ) <= model.q
model.waiting_area = pyo.Constraint(model.T, rule=waiting_area_rule)

# --------------------------------------------------------
# 6) Define Objective
# --------------------------------------------------------
# Minimize:
#   sum_{k,t,m} (c_k * y_{k,t,m} + c'_k * z_{k,t,m})
#   + sum_{i,o,t} (d_o - t)*h_o * e_{i,o,t}
def objective_rule(model):
    vehicle_cost = sum(model.c_owned[k]*model.y[k, t, m] + model.c_rented[k]*model.z[k, t, m]
                       for k in model.K for t in model.T for m in model.M)
    earliness_cost = sum((model.d_due[o] - t)*model.h[o]*model.e[i, o, t]
                         for i in model.I
                         for o in model.O
                         for t in model.T
                         # typically you'd restrict to t <= d_due[o] if you only penalize earliness
                         if t <= model.d_due[o])
    return vehicle_cost + earliness_cost

model.Obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)


In [ ]:

# --------------------------------------------------------
# 7) Solve
# --------------------------------------------------------
# In practice, you will load/initialize data for sets and parameters, 
# then call a solver such as CBC, GLPK, Gurobi, or CPLEX:
#    solver = pyo.SolverFactory('gurobi')  # or cplex, glpk, etc.
#    solver.solve(model, tee=True)

# Example: 
# solver = pyo.SolverFactory('glpk')
solver= pyo.SolverFactory('cplex')
results = solver.solve(model, tee=True)
results.write()

# At this point, you can query solution values:
for i in model.I:
    for k in model.K:
        for t in model.T:
            for m in model.M:
                for s in model.S:
                    if pyo.value(model.x[i,k,t,m,s]) > 0.5:
                        print(f"Pallet {i} shipped by vehicle {k} on day {t}, trip {m}, size {s}")



Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.6.2.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2015.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\alt_user\AppData\Local\Temp\tmpea8qjyym.cplex.log' open.
CPLEX> Problem 'C:\Users\alt_user\AppData\Local\Temp\tmpp2ivy0fd.pyomo.lp' read.
Read time = 0.05 sec. (7.24 ticks)
CPLEX> Problem name         : C:\Users\alt_user\AppData\Local\Temp\tmpp2ivy0fd.pyomo.lp
Objective sense      : Minimize
Variables            :  101850  [Binary: 38304,  General Integer: 63546]
Objective nonzeros   :   24890
Linear constraints   :    3122  [Less: 2331,  Greater: 300,  Equal: 491]
  Nonzeros           :  314837
  RHS nonzeros       :     547

Variables            : Min LB: 0.0000000        Max UB: 1.

In [ ]:
# At this point, you can query solution values:
sumY=0
sumZ=0
sumE=0
for k in model.K:
    for t in model.T:
        for m in model.M:
                # print(f"{pyo.value(model.y[k,t,m])}")
                sumY+=pyo.value(model.y[k,t,m])
                sumZ+=pyo.value(model.z[k,t,m])
                sumE+=pyo.value(model.e[i, o, t])
                # if pyo.value(model.y[k,t,m]) > 0.5:
                    # print(f"Pallet {i} shipped by vehicle {k} on day {t}, trip {m}, size {s}")


for i in model.I:
    for o in model.O:
        for t in model.T:
            sumE+=pyo.value(model.e[i, o, t])      